## 🔧 Setup and Imports

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from ml_models.lstm_predictor import LSTMPredictor, MultiResourceLSTM
from ml_models.data_preprocessing import WorkloadDataLoader

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports completed successfully!")

## 📥 Load Training Data

In [ ]:
# Load preprocessed data
data_loader = WorkloadDataLoader()
workload_data = data_loader.generate_synthetic_workload(
    num_hosts=10,
    num_timesteps=1000,
    pattern='mixed'
)

# Select one host for detailed analysis
host_0_data = workload_data[:, 0]

# Train/test split
train_size = int(0.8 * len(host_0_data))
train_data = host_0_data[:train_size]
test_data = host_0_data[train_size:]

print(f"Total data points: {len(host_0_data)}")
print(f"Training data: {len(train_data)} points")
print(f"Test data: {len(test_data)} points")

# Visualize train/test split
plt.figure(figsize=(15, 4))
plt.plot(range(len(train_data)), train_data, label='Training Data', color='blue', alpha=0.7)
plt.plot(range(len(train_data), len(host_0_data)), test_data, label='Test Data', color='orange', alpha=0.7)
plt.axvline(train_size, color='red', linestyle='--', label='Train/Test Split')
plt.title('Host 0 CPU Utilization - Train/Test Split', fontsize=14, fontweight='bold')
plt.xlabel('Timestep')
plt.ylabel('Utilization (%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 🏋️ Train LSTM Predictor

In [ ]:
# Initialize LSTM predictor
lstm_predictor = LSTMPredictor(
    sequence_length=10,
    lstm_units=64,
    dropout_rate=0.2
)

# Train the model
print("Training LSTM model...")
history = lstm_predictor.train(
    data=train_data,
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

print("\n✅ Training completed!")

## 📊 Training Progress Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss curve
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_title('Model Loss During Training', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE curve
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_title('Mean Absolute Error During Training', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/graphs/lstm_training_progress.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")

## 🔮 Make Predictions on Test Data

In [ ]:
# Generate predictions
predictions = []
actual_values = []

# Use sliding window over test data
for i in range(lstm_predictor.sequence_length, len(test_data)):
    sequence = test_data[i - lstm_predictor.sequence_length:i]
    pred = lstm_predictor.predict(sequence)
    predictions.append(pred[0])
    actual_values.append(test_data[i])

predictions = np.array(predictions)
actual_values = np.array(actual_values)

print(f"Generated {len(predictions)} predictions on test data")

## 📈 Prediction Accuracy Evaluation

In [ ]:
# Calculate metrics
mse = mean_squared_error(actual_values, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actual_values, predictions)
r2 = r2_score(actual_values, predictions)
mape = np.mean(np.abs((actual_values - predictions) / actual_values)) * 100

print("\n📊 Prediction Accuracy Metrics:")
print("=" * 50)
print(f"Mean Squared Error (MSE):     {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE):    {mae:.4f}")
print(f"R² Score:                     {r2:.4f}")
print(f"Mean Absolute Percentage Error: {mape:.2f}%")
print("=" * 50)

## 📉 Prediction vs Actual Visualization

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 1. Time series comparison
timesteps = range(len(predictions))
axes[0].plot(timesteps, actual_values, label='Actual', linewidth=2, alpha=0.7)
axes[0].plot(timesteps, predictions, label='Predicted', linewidth=2, alpha=0.7, linestyle='--')
axes[0].set_title('LSTM Predictions vs Actual Values', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Timestep')
axes[0].set_ylabel('CPU Utilization (%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Scatter plot
axes[1].scatter(actual_values, predictions, alpha=0.5, s=30)
axes[1].plot([actual_values.min(), actual_values.max()], 
             [actual_values.min(), actual_values.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_title('Actual vs Predicted - Scatter Plot', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Actual Utilization (%)')
axes[1].set_ylabel('Predicted Utilization (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Add R² annotation
axes[1].text(0.05, 0.95, f'R² = {r2:.4f}', 
             transform=axes[1].transAxes, 
             fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('../results/graphs/lstm_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

## 🎯 Prediction Error Analysis

In [ ]:
# Calculate prediction errors
errors = actual_values - predictions

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 1. Error distribution
axes[0].hist(errors, bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0].axvline(errors.mean(), color='blue', linestyle='--', linewidth=2, label=f'Mean Error: {errors.mean():.2f}')
axes[0].set_title('Prediction Error Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Error (Actual - Predicted)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Error over time
axes[1].plot(timesteps, errors, color='red', alpha=0.6)
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].fill_between(timesteps, 0, errors, where=(errors > 0), color='red', alpha=0.3, label='Underestimation')
axes[1].fill_between(timesteps, 0, errors, where=(errors < 0), color='green', alpha=0.3, label='Overestimation')
axes[1].set_title('Prediction Error Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Timestep')
axes[1].set_ylabel('Error')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/graphs/lstm_error_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nError Statistics:")
print(f"Mean Error: {errors.mean():.4f}")
print(f"Std Dev: {errors.std():.4f}")
print(f"Min Error: {errors.min():.4f}")
print(f"Max Error: {errors.max():.4f}")

## 📈 Trend Detection Analysis

In [ ]:
# Test trend detection
trend_predictions = []

for i in range(lstm_predictor.sequence_length, len(test_data)):
    sequence = test_data[i - lstm_predictor.sequence_length:i]
    trend = lstm_predictor.predict_trend(sequence)
    trend_predictions.append(trend)

# Count trend occurrences
from collections import Counter
trend_counts = Counter(trend_predictions)

print("\n📊 Trend Detection Results:")
print("=" * 50)
for trend, count in trend_counts.items():
    percentage = (count / len(trend_predictions)) * 100
    print(f"{trend.upper()}: {count} ({percentage:.2f}%)")
print("=" * 50)

# Visualize trend distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Pie chart
colors = {'increasing': '#ff9999', 'stable': '#66b3ff', 'decreasing': '#99ff99'}
trend_colors = [colors[trend] for trend in trend_counts.keys()]
axes[0].pie(trend_counts.values(), labels=trend_counts.keys(), autopct='%1.1f%%', 
            colors=trend_colors, startangle=90)
axes[0].set_title('Workload Trend Distribution', fontsize=14, fontweight='bold')

# Trend over time
trend_map = {'increasing': 1, 'stable': 0, 'decreasing': -1}
trend_numeric = [trend_map[t] for t in trend_predictions]
axes[1].plot(trend_numeric, linewidth=1, color='purple')
axes[1].set_title('Detected Trends Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Timestep')
axes[1].set_ylabel('Trend (-1: Decreasing, 0: Stable, 1: Increasing)')
axes[1].set_yticks([-1, 0, 1])
axes[1].set_yticklabels(['Decreasing', 'Stable', 'Increasing'])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/graphs/lstm_trend_detection.png', dpi=300, bbox_inches='tight')
plt.show()

## 🔁 Multi-Resource LSTM (CPU + RAM)

In [ ]:
# Generate multi-resource data
cpu_data = workload_data[:, 0]
ram_data = workload_data[:, 1]

# Train multi-resource predictor
multi_predictor = MultiResourceLSTM(
    sequence_length=10,
    lstm_units=64,
    num_resources=2
)

print("Training Multi-Resource LSTM...")
multi_history = multi_predictor.train(
    cpu_data=cpu_data[:train_size],
    ram_data=ram_data[:train_size],
    epochs=30,
    batch_size=32
)

print("\n✅ Multi-Resource LSTM training completed!")

# Make predictions
cpu_pred, ram_pred = multi_predictor.predict(
    cpu_data[train_size-10:train_size],
    ram_data[train_size-10:train_size]
)

print(f"\nPredicted CPU Utilization: {cpu_pred[0]:.2f}%")
print(f"Predicted RAM Utilization: {ram_pred[0]:.2f}%")

## 💾 Save Trained Model

In [ ]:
# Save models
os.makedirs('../models', exist_ok=True)

lstm_predictor.save('../models/lstm_predictor_host0.h5')
multi_predictor.save('../models/multi_resource_lstm.h5')

print("✅ Models saved successfully!")
print("   - lstm_predictor_host0.h5")
print("   - multi_resource_lstm.h5")

## 📋 Summary

**Key Results:**
- ✅ LSTM model trained successfully with low validation loss
- ✅ Prediction accuracy: R² = {r2:.4f}, MAE = {mae:.4f}
- ✅ Trend detection identifies workload patterns effectively
- ✅ Multi-resource prediction supports CPU + RAM forecasting
- ✅ Models saved for integration with DQN agent

**Next Steps:**
1. Integrate LSTM predictions with DQN agent (see `03_dqn_training.ipynb`)
2. Evaluate full LSTM-DQN hybrid system
3. Compare with baseline algorithms